Makine öğrenmesiyle tüketici şikayetlerinin sınıflandırılması, müşteri hizmetleri departmanlarının günlük olarak aldığı binlerce şikayeti daha etkili bir şekilde yönetmelerine yardımcı olur. Bu süreç, şikayetlerin doğasına göre kategorize edilmesini sağlayarak, öncelikli olarak çözülmesi gereken sorunların belirlenmesine olanak tanır.

## Projenin Amacı

Bu proje, tüketici şikayetlerini makine öğrenmesi kullanarak sınıflandırmayı hedefler. Bu sayede, müşteri hizmetleri ekipleri, hangi şikayetlerin acil müdahale gerektirdiğini hızlıca belirleyebilir ve kaynaklarını daha verimli kullanabilir.

## Kullanılan Veri Kümesi

Proje kapsamında kullanılan veri kümesi, tüketiciler tarafından bildirilen şikayetleri içerir. Bu veri kümesinde şikayetlerin türü, ilgili ürün veya hizmet, yaşanan sorun ve şikayetin detaylı açıklaması gibi bilgiler bulunmaktadır. Veri kümesinin boyutu oldukça büyük olduğundan, Google Drive'a yüklenip Google Colab üzerinde işlenmesi önerilmektedir.

## Veri İşleme ve Modelleme Adımları

Veri Yükleme ve Temizleme: Veri kümesi pandas kütüphanesi kullanılarak yüklenir. Gereksiz sütunlar kaldırılır ve eksik veriler kontrol edilir.

Metin Ön İşleme: Doğal dil işleme (NLP) teknikleriyle metin verileri temizlenir. Bu adımda noktalama işaretleri, durdurma kelimeleri (stopwords) ve özel karakterler kaldırılır.

Özellik Çıkarımı: Metin verileri sayısal verilere dönüştürülür. Bu işlem için genellikle CountVectorizer veya TF-IDF (Term Frequency-Inverse Document Frequency) yöntemleri kullanılır.

Model Eğitimi: Elde edilen özellikler kullanılarak makine öğrenmesi modelleri eğitilir. Bu projede Naive Bayes ve Stokastik Gradyan İnişi (SGD) sınıflandırıcıları kullanılmıştır.

Model Değerlendirmesi: Eğitilen modellerin performansı doğruluk, kesinlik ve geri çağırma gibi metriklerle değerlendirilir.

## Sonuç


Makine öğrenmesiyle tüketici şikayetlerinin sınıflandırılması, müşteri hizmetleri süreçlerini otomatikleştirerek daha hızlı ve etkili bir hizmet sunulmasını sağlar. Bu yaklaşım, şirketlerin müşteri memnuniyetini artırmalarına ve operasyonel verimliliklerini geliştirmelerine yardımcı olur.

Projenin detaylarına ve kodlarına aşağıdaki bağlantıdan ulaşabilirsiniz:
[Consumer Complaint Classification with Machine Learning](https://amanxai.com/2022/11/28/consumer-complaint-classification-with-machine-learning/)


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import nltk
import re
from nltk.corpus import stopwords
import string

data = pd.read_csv("consumercomplaints.csv")
print(data.head())

   Unnamed: 0 Date received  \
0           0    2022-11-11   
1           1    2022-11-23   
2           2    2022-11-16   
3           3    2022-11-15   
4           4    2022-11-07   

                                             Product  \
0                                           Mortgage   
1  Credit reporting, credit repair services, or o...   
2                                           Mortgage   
3                        Checking or savings account   
4                                           Mortgage   

                  Sub-product                           Issue  \
0  Conventional home mortgage  Trouble during payment process   
1            Credit reporting     Improper use of your report   
2                 VA mortgage  Trouble during payment process   
3            Checking account             Managing an account   
4      Other type of mortgage  Trouble during payment process   

                                       Sub-issue  \
0                                

In [7]:
data = pd.read_csv("consumercomplaints.csv")

In [8]:
data = data.drop("Unnamed: 0",axis=1)

In [9]:
print(data.isnull().sum())

Date received                         0
Product                               0
Sub-product                      235294
Issue                                 0
Sub-issue                        683355
Consumer complaint narrative    1987977
dtype: int64


In [10]:
data = data.dropna()


In [11]:
print(data["Product"].value_counts())

Product
Credit reporting, credit repair services, or other personal consumer reports    507582
Debt collection                                                                 192045
Credit card or prepaid card                                                      80410
Checking or savings account                                                      54192
Student loan                                                                     32697
Vehicle loan or lease                                                            19874
Payday loan, title loan, or personal loan                                         1008
Name: count, dtype: int64


## Training Consumer Complaint Classification Model

In [ ]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Consumer complaint narrative"] = data["Consumer complaint narrative"].apply(clean)

In [13]:
data = data[["Consumer complaint narrative", "Product"]]
x = np.array(data["Consumer complaint narrative"])
y = np.array(data["Product"])

cv = CountVectorizer()
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [14]:
sgdmodel = SGDClassifier()
sgdmodel.fit(X_train,y_train)

SGDClassifier()

In [15]:
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = sgdmodel.predict(data)
print(output)

Enter a Text:  Investigation took more than 30 days and nothing was changed when clearly there are misleading, incorrect, inaccurate items on my credit report..i have those two accounts attached showing those inaccuracies... I need them to follow the law because this is a violation of my rights!! The EVIDENCE IS IN BLACK AND WHITE ....


['Credit reporting, credit repair services, or other personal consumer reports']


Enter a Text:
Investigation took more than 30 days and nothing was changed when clearly there are misleading, incorrect, inaccurate items on my credit report..i have those two accounts attached showing those inaccuracies... I need them to follow the law because this is a violation of my rights!! The EVIDENCE IS IN BLACK AND WHITE ....